In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import datetime
import re
import urllib
from bs4 import BeautifulSoup
from random import randint
import pyLDAvis.gensim
from pprint import pprint

In [3]:
initial_date_str = '1 Jan 2018'
initial_date = datetime.datetime.strptime(initial_date_str, '%d %b %Y')
final_date_str = '13 Jun 2018'

final_date = datetime.datetime.strptime(final_date_str, '%d %b %Y')
current_date = initial_date

In [4]:
tweets_list = []
while current_date < final_date :        
    aux_list = open('Tweets/' + current_date.strftime("%Y-%m-%d") + '_trump.txt',encoding='UTF-8').read().split('\n')
    aux_list = [current_date.strftime("%Y-%m-%d") + ', ' + row for row in aux_list]
    tweets_list.extend(aux_list)
    current_date = current_date + datetime.timedelta(days=1)

In [5]:
tweets_list[6500]

'2018-04-15, JTStevison, https://twitter.com/JTStevison/status/985656472687259649, 1, 11, 4, Perfect. <img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><img class="Emoji Emoji--forText" src="https://abs.twimg.com/emoji/v2/72x72/1f602.png" draggable="false" alt="😂" title="Face with tears of joy" aria-label="Emoji: Face with tears of joy"><a href="/hashtag/DonaldTrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b><strong>DonaldTrump</strong></b></a> <a href="/hashtag/Syria?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Syria</b></a> <a href="/hashtag/SyrianStrikes?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>SyrianStrikes</b></a> <a href="/hashtag/Trump?src=hash" data-qu

In [6]:
def add_space_between_a_tags(html):
    pos_1 = html.find('<a')
    while pos_1 != -1:
        html = html[:pos_1] + ' ' + html[pos_1:]
        pos_1 = html.find('<a',pos_1+2)
        
    pos_1 = html.find('</a>')
    while pos_1 != -1:
        html = html[:pos_1+4] + ' ' + html[pos_1+4:]
        #if len(html) == html.find('</a>',pos_1 + 4)
        pos_1 = html.find('</a>',pos_1+4)
        
    return html

In [7]:
print(add_space_between_a_tags(tweets_list[9870]))

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This  <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a>   <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a>   <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a>   <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a>   <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a>   <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link j

In [8]:
print(tweets_list[9870])

2018-06-09, reggaerapstarCP, https://twitter.com/reggaerapstarCP/status/1005393266530021379, 0, 1, 3, Remember this?This This <a href="/hashtag/Original?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>Original</b></a> <a href="/hashtag/hand?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>hand</b></a> <a href="/hashtag/painting?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>painting</b></a> <a href="/hashtag/handdesigned?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>handdesigned</b></a> <a href="/hashtag/usa?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir="ltr"><s>#</s><b>usa</b></a> <a href="/hashtag/presidenttrump?src=hash" data-query-source="hashtag_click" class="twitter-hashtag pretty-link js-nav" dir=

In [9]:
def get_hashtags(tweet):
    hashtags = []
    pos = tweet.find('#')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        hashtags.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('#',pos + 1)
    return hashtags

In [10]:
print(get_hashtags("Body Language Analysis No.4303: Donald Trump, G7 Summit, Women's Empowerment, and Rage -  #Nonverbal &  #EmotionalIntelligence •   •  #DonaldTrump #AngelaMerkel #Anger #BehaviorAnalysis #BodyLanguage #BodyLanguageExpert #Analysis #EQ #G7 #G7Summit"))

['#Nonverbal', '#EmotionalIntelligence', '#DonaldTrump', '#AngelaMerkel', '#Anger', '#BehaviorAnalysis', '#BodyLanguage', '#BodyLanguageExpert', '#Analysis', '#EQ', '#G7']


In [11]:
def get_mentions(tweet):
    mentions = []
    pos = tweet.find('@')
    while pos != -1:
        pos_aux = tweet.find(" ",pos)
        if pos_aux == -1:
            break
        mentions.append(tweet[pos:pos_aux])
        tweet = tweet[:pos] + ' ' + tweet[pos_aux:]
        pos = tweet.find('@',pos + 1)
    return mentions

In [12]:
print(get_mentions("May God Bless you and Bless God for  #DonaldTrump @lindathomas102 @louisa_ip @maga_417 @MarcGriff89 @MarjorieLoy1@Maximus_Paulus @MC_Mad_Man @mgw81180 @MrSteveRangel @MyMountainTop @Namashary @netteKnows @NotUnderdog @NYKchi88 @nykon187@ofarther"))

['@lindathomas102', '@louisa_ip', '@maga_417', '@MarcGriff89', '@MarjorieLoy1@Maximus_Paulus', '@MC_Mad_Man', '@mgw81180', '@MrSteveRangel', '@MyMountainTop', '@Namashary', '@netteKnows', '@NotUnderdog', '@NYKchi88']


In [13]:
dates = []
user = []
status = []
comments = []
likes = []
retweets = []
tweet = []
tweet_no_html = []
hashtags = []
mentions = []
for tweet_status in tweets_list:
    pos = tweet_status.find(',')
    dates.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    user.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    status.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    comments.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    likes.append(tweet_status[:pos])
    
    tweet_status = tweet_status[pos + 1:].strip()
    pos = tweet_status.find(',')
    retweets.append(tweet_status[:pos])
        
    tweet_status = tweet_status[pos + 1:].strip()    
    soup = BeautifulSoup(add_space_between_a_tags(tweet_status),'lxml')
    
    tweet_status_no_html = soup.text
    
    tweet.append(tweet_status_no_html)
    
    pos_2 = tweet_status_no_html.find('pic.twitter.com/')
    tweet_status_no_html = tweet_status_no_html[:pos_2].strip()
    pos_3 = tweet_status_no_html.find('http')
    while pos_3 != -1:        
        tweet_status_no_html = tweet_status_no_html[:pos_3 - 1] + ' ' + tweet_status_no_html[pos_3:]
        pos_3 = tweet_status_no_html.find('http',pos_3 + 1)
    tweet_status_no_html = re.sub(r"http\S+", "", tweet_status_no_html)
    tweet_status_no_html = tweet_status_no_html.strip().replace("\xa0","")
    tweet_no_html.append(tweet_status_no_html)
    
    hashtags.append(get_hashtags(tweet_status_no_html))
    mentions.append(get_mentions(tweet_status_no_html))

In [14]:
dates[1]

'2018-01-01'

# Removing short tweets

In [15]:
count = 0
index = 0
for t in tweet_no_html:
    if len(t.lower().replace("donaldtrump","")) < 20:# or int(retweets[index]) == 0 or int(likes[index]) == 0 or int(comments[index]) == 0:
        del(dates[index])
        del(user[index])
        del(status[index])
        del(comments[index])
        del(likes[index])
        del(retweets[index])
        del(tweet[index])
        del(tweet_no_html[index])
        del(hashtags[index])
        del(mentions[index])
        count = count + 1
    index = index + 1
print(count)
    

253


In [16]:
x = randint(0, len(dates))
print(x)
print(dates[x],user[x],status[x],comments[x],likes[x],retweets[x],tweet[x],tweet_no_html[x],hashtags[x],mentions[x],end="\n")

7901
2018-05-11 MerlinYYC https://twitter.com/MerlinYYC/status/994939601528729601 0 5 6 #JamesComey  says  #Trump  presidency is like a 'forest fire' you can listen to whole interview hear  @TheCurrentCBC http://www.cbc.ca/radio/thecurrent/the-current-for-may-11-2018-1.4657253/james-comey-says-trump-s-presidency-is-like-a-forest-fire-but-after-that-comes-a-chance-for-regrowth-1.4657257 … @mmpadellan @CNN #DonaldTRUMP  #JamesComey  says  #Trump  presidency is like a 'forest fire' you can listen to whole interview hear  @TheCurrentCBC … @mmpadellan @CNN #DonaldTRUMP ['#JamesComey', '#Trump'] ['@TheCurrentCBC', '@mmpadellan', '@CNN']


# Grouping by day

In [17]:
dates_by_day = []
user_by_day = []
status_by_day = []
comments_by_day = []
likes_by_day = []
retweets_by_day = []
tweet_by_day = []
tweet_no_html_by_day = []
hashtags_by_day = []
mentions_by_day = []

i = 0
index = 0
current_date = dates[0]
flag = True
for d in dates:
    if (index == 0 and flag) or dates[i] != current_date:
        dates_by_day.append([dates[i]])
        user_by_day.append([user[i]])
        status_by_day.append([status[i]])
        comments_by_day.append([comments[i]])
        likes_by_day.append([likes[i]])
        retweets_by_day.append([retweets[i]])
        tweet_by_day.append([tweet[i]])
        tweet_no_html_by_day.append([tweet_no_html[i]])
        hashtags_by_day.extend([hashtags[i]])
        mentions_by_day.extend([mentions[i]])        
        
        if i != 0:
            index = index + 1
        current_date = dates[i]
        flag = False
    else:
        #print(tweets,index)
        #dates_by_day.append([dates[i]])
        user_by_day[index].append(user[i])
        status_by_day[index].append(status[i])
        comments_by_day[index].append(comments[i])
        likes_by_day[index].append(likes[i])
        retweets_by_day[index].append(retweets[i])
        tweet_by_day[index].append(tweet[i])
        tweet_no_html_by_day[index].append(tweet_no_html[i])
        hashtags_by_day[index].extend(hashtags[i])
        mentions_by_day[index].extend(mentions[i]) 
    i = i + 1
    

In [18]:
tweet_no_html_by_day[0]

['#America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA',
 '#DonaldTrump  stop blaming PAK... and PAK Should  #StopAmericanSupplies',
 'Pakistan will Not Do More !!  #NoMoreTrump #DonaldTrump #StopAmericanSupplies',
 '#DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?',
 'Pakistan is not Fighting for Money : DG ISPR #NoMoreTrump #StopAmericanSupplies #DonaldTrump',
 'Dear  #DonaldTrump  you know fairly after March 2018.......you will never be able to threatened State of Pakistan ever again.......So now  #NoMoreTrump',
 'So, the President of America doesn\'t know that the exeggerated figure of $33 billion in Coalition Support Fund "IS NOT AID".  #DonaldTrump',
 "Only problem is .... #DonaldTrump  can't be disappeare",
 '#DonaldTrump  is trending top in  #Pakistan  right now and not for any good reason',
 'Becoz we are busy in lifting "Mukka" from Mukka chawk..all 33 billion dollars are used to vanish MQM frm Karachiwe need more coz MQM is still the

In [19]:
index = 0
for i in tweet_no_html:
    print(index,':',i)
    index = index + 1

0 : #America  demands  #TrumpShowUsYourPenis #DonaldTrump #MAGA
1 : #DonaldTrump  stop blaming PAK... and PAK Should  #StopAmericanSupplies
2 : Pakistan will Not Do More !!  #NoMoreTrump #DonaldTrump #StopAmericanSupplies
3 : #DonaldTrump #DonaldTrumpsPak  Short term memory loss syndrome OR Bipolar disorder?
4 : Pakistan is not Fighting for Money : DG ISPR #NoMoreTrump #StopAmericanSupplies #DonaldTrump
5 : Dear  #DonaldTrump  you know fairly after March 2018.......you will never be able to threatened State of Pakistan ever again.......So now  #NoMoreTrump
6 : So, the President of America doesn't know that the exeggerated figure of $33 billion in Coalition Support Fund "IS NOT AID".  #DonaldTrump
7 : Only problem is .... #DonaldTrump  can't be disappeare
8 : #DonaldTrump  is trending top in  #Pakistan  right now and not for any good reason
9 : Becoz we are busy in lifting "Mukka" from Mukka chawk..all 33 billion dollars are used to vanish MQM frm Karachiwe need more coz MQM is still th

720 : . @washingtonpost   Thank you for getting it right on the bipartisan meeting with Senators   #DonaldTrump  who will always be a showboating con man, 3 Card Monte.  #JusticeforKate …
721 : ' #NaturalbornCitizen ' is One Thing  … #donaldtrump #seanhannity #michaelsavage #tuckercarlson
722 : #Israel  using electricity to blackmail  #Gaza  | The Electronic Intifada  … #BreakGazaSeige #OpenRafahBorder #FreePalestine  #Palestine #BoycottIsrael #BDS #CNN #BBC #FoxNews #ABC #NBC #CBS #DonaldTrump #NikkiHaley #Apartheid #Israel #Netanyahu
723 : #DonaldTrump  to attend Davos World Economic Forum, may meet PM  #NarendraModi …
724 : THE JIG IS UP !  @realDonaldTrump  Is Not Above The Law !  #DonaldTrump  Has To Answer All Questions To A Grand Jury. And The  #SteeleDossier  Is Real & True. Steele Did The Right Thing Going To U.S Investigators It Was A National Security Crimes.  #TrumpRussia  …
725 : Student activist sentenced to 12 months in  #Israeli  jail  … #HandsOffAlquds #Jerusalem #Free

785 : I love  #DonaldTrump  more very day #ICE #7Eleven …
786 : Yoohoo!!  #DonaldTrump ! You got some 'splainin' to do!!  You do realize that we already know this? You fool no one but yourself, buddy.  #EpicFail ‘Stable genius’ Donald Trump doesn’t actually have an MBA from Wharton: report | Raw Story    via rawstor
787 : #DonaldTrump  blasts  #immigration  ruling, calls U.S. court system 'broken and unfair'   @micarosenberg
788 : Is he willing to testify about  #DonaldTrump 's drug-running connections?  …
789 : Keep it moving! DELETE it! then RESET it!   #Symply2018 #WorstFirstDate #AnnoysMeMoreThanAnything #DonaldTrump
790 : They are laughing at you.  #DonaldTrump
791 : Homeland Security Agents Raid 20 ‘Maternity Hotels’ in Calif. -  … #dobbs #seanhannity #tuckercarlson #donaldtrump #foxnews #michaelsavage #anchorbabies #rushlimbaugh
792 : Helping  @joelycett  take down  @realDonaldTrump  one tweet at a time.  #stupidorangeberkweasel #DonaldTrump
793 : . @Oprah  or  @realDonaldTrump 

1118 : Oh good Lord! Donald Trump holds an umbrella over himself as he, Melania and Barron board Air Force One and leaves wife and child in the cold.  #DonaldTrump …
1119 : Nicole Kidman Says: Trump Is The Best President In U.S History, We Must Respect Him. Do you support her? … #realDonaldTrump #PresidentTrump #NicoleKidman #MAGA #TheAmericans #Trump #DonaldTrump #MakeAmericaGreatAgain
1120 : Senator Flake,  #FlakeyFlake  attacks Trump for using the term  #FakeNews  Jeff Flake compares the  #DonaldTrump #FakeNews  comments to those of communist leader Joseph Stalin OH REALLY? This guy would make a great Democrat. What a  #Flake  …
1121 : #KathrynBigelow  targets  #DonaldTrump  in acceptance statement
1122 : "Frankentrump, tweet monster" -  #donaldtrump
1123 : Room Full Of Constituents Erupt In Laughter After GOP Senator Tries To Defend Trump (VIDEO)  #DonaldTrump #gop …
1124 : #DonaldTrump  is a liar, racist and a crook.  #maddow  Link to Buzzfeed story: …
1125 : If  #DonaldTrump ’s a

1618 : 70% who Strongly Approve of the job  #PresidentTrump  is doing think a  #specialprosecutor  is needed to investigate the  #FBI . Among those who Strongly Disapprove of the president’s job performance, only 36% agree.   @POTUS #HillaryClinton #DonaldTrump
1619 : Mueller appears to be zeroing in on obstruction questions - NBC NewsThere is more than one path to  #Trump , let’s not forget that. #DemForce #TrumpRussiaConspiracy #TrumpRussia #DonaldTrump #MuellerTime #Mueller #WedsdayWisdom #TheResistance #BlueWave2018 …
1620 : In 1973, White House Counsel  #JohnDean  told Richard Nixon, "A cancer is growing on the presidency." In 2018,  #DonaldTrump  IS the cancer.   #WednesdayWisdom #ResignNow
1621 : 62% of  #GOP  are calling for an outside prosecutor to investigate the  #FBI , as is a plurality (49%) of unaffiliated voters. Among  #Democrats , 38% favor a  #specialprosecutor ; 40% are opposed, but 22% are undecided.   #HillaryClinton #DonaldTrump
1622 : The  #WomenMarch  showed its

...

5284 : ICYMI, here are all the highlights from  #StormyDaniels '  #60Minutes  interview about  #DonaldTrump :
5285 : What did you think about  #StormyDaniels ' interview about  #DonaldTrump  on  #60Minutes ??
5286 : #DonaldTrump  doesn't wear condoms when he cheats on his 3rd wife. Also, he has yet to refute  #StormyDaniels  via twitter. Donald tRump has attached the Pope via twitter, but not Putin or Stormy Danielle's. Interesting
5287 : Lawyers  #DonaldTrump  has contacted:
5288 : My daily reminder that we are facing an authoritarian and constitutional crisis.  #DonaldTrump #trump #Mueller #houston #trumprussia #freedom
5289 : After watching the  #60Minutes  interview with  #stormydaniels , do you think she would actually be a better president than  #donaldtrump
5290 : Is there anything immoral illegal and despicable  #DonaldTrump  didn’t do to win the election?  #NotMyPresident
5291 : I was threatened to keep quiet, says  #StormyDaniels  on  #DonaldTrump #affair Read  @ANI  Story | 

5450 : NEW EPISODE:  @theqnow  - Movie Magic @DenzelPitt  &  @ksudodger  join the show to discuss the greatest comedic movie of the past 20 years! Did yours make the list, find out. #DonaldTrump #tbt #movies #bestus #StormyDaniels DOWNLOAD: …
5451 : It’s creepy the way  #MikePence  stares at  #DonaldTrump ! I wonder if his Pence’s wife and mom get jealous of the way he stars lovingly at Trump!  #SpankMeTrump  is what he’s thinking
5452 : Scientists have discovered a new species of moth which they have named after Donald Trump. It's called "Neopalpa donaldtrumpi" and has golden scales resembling his hair and a small penis!  #DonaldTrump #science #nature #fact
5453 : Congress just killed workplace protections for underpaid minor league baseball players. Notch another win for the 1% under Republicans and  #DonaldTrump .
5454 : Reading  #DonaldTrump 's mind. We are following  #WilliamSafire 's take on what the  #president  is thinking.  #Trump  thinks, I can do and say whatever I want,  #R

6617 : Watch  #Conservative  walks into  #Starbucks  poses as  #liberal ,  #demands  “free  #coffee  for  #reparations ”..- & gets it! Starbucks accepts  #RaceCard  as valid form of payment now. Go get your free coffee.  #ThursdayThoughts #FridayFeeling #DonaldTrump
6618 : When Olivia started bleeding out Bc I don’t Stan a trump supporter on a show intended to be filled with embracing diversity  #ronnihawk #DonaldTrump #ronnihawktrump #onmyblock #netflix #olivia #recastolivia …
6619 : MLB5* MLB [959]ORIOLES (BALTIMORE) -120 [A COBB -R/J ZIMMERMANN -R]Bet and Win! #FreePicks #NYCSP #BetAndWin #TheOriginals #PuertoRico #ThrowbackThursday  #SJSharks   #Comey #Spurs #DonaldTrump #MLB #MLBPicks #SportsBetting #Betonline #BaltimoreOrioles #Orioles
6620 : People in  #California  seem very happy. They have a booming  #economy , great food,  #Progressive  laws and a diverse population. It has problems, the biggest being the crook "infesting" the  #WhiteHouse  - however, that is solvable. #BlueW

7950 : circumvent anymore negative  #leaks  or testimonies coming out about  #DonaldTrump  and his crime ring. So, Mike suggested using his  #Q  level authority in the  #intelligencecommunity  to distract from the negative  #leaks
7951 : After electing  #DonaldTrump , I really don't think the  #US  is in a place to analyze  #IraqiElections2018 , claiming it will lean one way or another. …
7952 : My Trumpster sister told me today to stop offending people online. My head exploded. #KellySadler #OliverNorth #MikeKelly #ThomasMcInerney #MikePence #JohnKelly #SarahSanders #DonaldTrump #RudyGiuliani #SeanHannity #AnnCoulter #LauraIngraham #Melania #Ivanka #outofcharacters
7953 : #DonaldTrump  owes his workers $5 million in unpaid wages. That’s only part of what he owes!  …
7954 : How  #RobertMueller  knew  #Ford  Motor Company rejected  #MichaelCohen ’s request for a  #DonaldTrump  bribe  …  via  @PalmerReport
7955 : we are NOT talking about  #satanists ,  #wiccans ,  #shamans , or various f

9283 : #DidYouKnow #donaldtrump  has always been a problem for the  #NFL  - He even attempted a $1.7 billion lawsuit.
9284 : Prema-donna million dollar athletes not gonna bow down before anyone as they have proven on field of play.  Glad to see  #DonaldTrump  put them in their place when no one else will.  Sorry ‘sports figures’ but you are idiots for not honoring those who gave all for freedom of speech
9285 : President  #DonaldTrump  said he had the "absolute right" to pardon himself, and as criticism mounted,  #KellyanneConway  came to his defence
9286 : How  #DonaldTrump  is emerging as a big threat to the  #dollar  |  @Chellaney  |
9287 : #DonaldTrump  is a 5 Time Draft Dodger and a proven Pathological Liar.  Who is he to judge someone else’s  patriotism?  #CadetBoneSpurs
9288 : Donald Trump Cancels Philadelphia Eagles White House Visit.  #PhiladelphiaEagles #DonaldTrump #NFL
9289 : If  #DonaldTrump ,  #PaulManafort  and the people on his campaign are innocent, they wouldn’t behav

In [24]:
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from string import digits
import mpld3
import numpy as np
import pandas as pd
nltk.download('punkt')
stemmer = SnowballStemmer('english')
stopword = nltk.corpus.stopwords.words('english')
stopword = stopword + ['trump','donaldtrump','donald','realdonaldtrump']

[nltk_data] Downloading package punkt to /home/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [29]:
totalvocab_stemmed = []
totalvocab_tokenized = []
tweets_day = []
for t in tweet_no_html_by_day:
    i = "".join(t)
    tweets_day.append(i)
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [30]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [37]:
print(vocab_frame.sample)

<bound method NDFrame.sample of                                                      words
america                                            america
demand                                             demands
trumpshowusyourpeni                   trumpshowusyourpenis
donaldtrump                                    donaldtrump
maga                                                  maga
donaldtrump                                    donaldtrump
stop                                                  stop
blame                                              blaming
pak                                                    pak
and                                                    and
pak                                                    pak
should                                              should
stopamericansuppliespakistan  stopamericansuppliespakistan
will                                                  will
not                                                    not
do                      

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=stopword,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(tweets_day) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 6.73 s, sys: 27.8 ms, total: 6.76 s
Wall time: 6.77 s
(163, 616)


In [32]:
terms = tfidf_vectorizer.get_feature_names()

In [33]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 1.08 s, sys: 0 ns, total: 1.08 s
Wall time: 1.08 s


In [24]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

In [25]:
documents = [[token for token in doc if token not in stoplist and not token.isdigit()] for doc in documents]

In [26]:
#documents

In [27]:
for doc in documents:
    doc.sort()

dictionary = corpora.Dictionary(documents)
dictionary.compactify()
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [28]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:10]]

[[('dear', 1)],
 [('america', 1), ('demands', 1), ('maga', 1), ('trumpshowusyourpenis', 1)],
 [('enough', 1)],
 [('blaming', 1), ('pak', 2), ('stop', 1), ('stopamericansupplies', 1)],
 [('stopamericansupplies', 1), ('nomoretrump', 1), ('pakistan', 1)],
 [('bipolar', 1),
  ('disorder', 1),
  ('donaldtrumpspak', 1),
  ('loss', 1),
  ('memory', 1),
  ('short', 1),
  ('syndrome', 1),
  ('term', 1)],
 [('stopamericansupplies', 1),
  ('nomoretrump', 1),
  ('pakistan', 1),
  ('fighting', 1),
  ('ispr', 1),
  ('money', 1)],
 [('dear', 1),
  ('nomoretrump', 1),
  ('pakistan', 1),
  ('able', 1),
  ('ever', 1),
  ('fairly', 1),
  ('know', 1),
  ('march', 1),
  ('never', 1),
  ('state', 1),
  ('threatened', 1)],
 [('america', 1),
  ('know', 1),
  ('aid', 1),
  ('billion', 1),
  ('coalition', 1),
  ('exeggerated', 1),
  ('figure', 1),
  ('fund', 1),
  ('president', 1),
  ('support', 1)],
 [('disappeare', 1), ('problem', 1)]]

In [29]:
'''lda_model =  models.ldamodel.LdaModel(corpus, num_topics=30)
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus,dictionary)'''

'lda_model =  models.ldamodel.LdaModel(corpus, num_topics=30)\npyLDAvis.enable_notebook()\npyLDAvis.gensim.prepare(lda_model, corpus,dictionary)'

In [30]:
lda_model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary)

In [31]:
topic_words = lda_model.show_topics(formatted=False)
print(topic_words[1])

(1, [('president', 0.010419962), ('kimjongun', 0.007426276), ('usa', 0.0068267905), ('trumpkimsummit', 0.006468086), ('canada', 0.006068152), ('north', 0.004786145), ('korea', 0.0043517095), ('people', 0.0037318051), ('justintrudeau', 0.0035605803), ('man', 0.0034741575)])


In [32]:
x=lda_model.show_topics(num_topics=12, num_words=5,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

0::['singapore', 'president', 'maga', 'potus', 'foxnews']
1::['president', 'kimjongun', 'usa', 'trumpkimsummit', 'canada']
2::['kimjongun', 'trumpkimsummit', 'singaporesummit', 'president', 'summit']

singapore president maga potus foxnews
president kimjongun usa trumpkimsummit canada
kimjongun trumpkimsummit singaporesummit president summit


In [33]:
topic_words_per_document.append(topic_words)

In [34]:
topic_words_per_document

[[(0,
   [('singapore', 0.012361183),
    ('president', 0.009280885),
    ('maga', 0.0071136784),
    ('potus', 0.00530841),
    ('foxnews', 0.0042490005),
    ('trumpkimsummit', 0.003925247),
    ('america', 0.003914129),
    ('body', 0.0037126432),
    ('cnn', 0.0036449907),
    ('analysis', 0.0035089129)]),
  (1,
   [('president', 0.010419962),
    ('kimjongun', 0.007426276),
    ('usa', 0.0068267905),
    ('trumpkimsummit', 0.006468086),
    ('canada', 0.006068152),
    ('north', 0.004786145),
    ('korea', 0.0043517095),
    ('people', 0.0037318051),
    ('justintrudeau', 0.0035605803),
    ('man', 0.0034741575)]),
  (2,
   [('kimjongun', 0.016140422),
    ('trumpkimsummit', 0.008114253),
    ('singaporesummit', 0.007860324),
    ('president', 0.0070596416),
    ('summit', 0.006467416),
    ('northkorea', 0.005222512),
    ('potus', 0.005012077),
    ('world', 0.0048545618),
    ('america', 0.0048336107),
    ('one', 0.004543916)])]]